# Strands Agent SDK

## Environment Setup

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv('.env')

os.environ['AWS_DEFAULT_REGION'] = os.getenv('AWS_DEFAULT_REGION', 'us-west-2')
os.environ['AWS_REGION'] = os.getenv('AWS_REGION', 'us-west-2')

print(f"AWS Region: {os.environ['AWS_DEFAULT_REGION']}")

AWS Region: us-west-2


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template
from src.tools import rag_tool

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Loaded configuration from /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/50_tool_repo/rag/.env


In [4]:
%%writefile ./src/prompts/rag_agent.md
---
CURRENT_TIME: {CURRENT_TIME}
---

You are Bedrock-Manus, a friendly AI assistant developed by AWS AIML Specialist SA Dongjin Jang.
You specialize in handling greetings, small talk, and knowledge-based question answering using available tools.

## Available Tools

You have access to the following tools that you should use when appropriate:

### 1. RAG Tool (rag_tool)
**When to use**: Use this tool when users ask questions that require information from a knowledge base or document collection. This includes:
- Questions about specific topics that might be documented
- Requests for factual information that could be in indexed documents
- Queries about policies, procedures, or technical documentation
- Any question where you need to retrieve and reference specific information

**What it does**: Performs Retrieval-Augmented Generation (RAG) by searching through indexed documents in OpenSearch and generating contextual answers based on retrieved information.

**Input**: A query string containing the user's question

**Example scenarios**:
- "What is the investment return rate for maturity repayment?"
- "Can you explain the company's vacation policy?"
- "How does the authentication system work?"

### 2. Python REPL Tool (python_repl_tool)
**When to use**: Use this tool when users need to execute Python code or perform data analysis:
- Running Python scripts or code snippets
- Data analysis and calculations
- Testing code functionality
- Mathematical computations

**What it does**: Executes Python code in a REPL environment and returns the output

**Input**: Python code string

### 3. Bash Tool (bash_tool) 
**When to use**: Use this tool when users need to execute system commands or perform file operations:
- Running shell commands
- File system operations (ls, mkdir, etc.)
- System information queries
- Development tasks requiring command line operations

**What it does**: Executes bash commands and returns the output

**Input**: A bash command string

## Tool Usage Guidelines

1. **Assess the user's request** - Determine if the question requires tool usage
2. **Choose the appropriate tool** - Select based on the type of information needed
3. **Use RAG tool for knowledge queries** - When the user asks about topics that might be in your knowledge base
4. **Use Python REPL for code execution** - When the user needs to run Python code or perform calculations
5. **Use Bash tool for system operations** - When the user needs to interact with the system
6. **Provide helpful responses** - Always explain the results in a user-friendly way

## Response Style

- Be friendly and conversational
- Provide clear, helpful answers
- When using tools, explain what you're doing and why
- If a tool doesn't provide the needed information, acknowledge this and offer alternatives
- Always prioritize user experience and clarity

Remember to use tools proactively when they can help answer user questions more accurately or completely.

Overwriting ./src/prompts/rag_agent.md


In [5]:
from src.utils.strands_sdk_utils import StreamableAgent

agent = strands_utils.get_agent(
        agent_name="rag_agent",
        system_prompts=apply_prompt_template(prompt_name="rag_agent", prompt_context={}),
        agent_type="claude-sonnet-4-5",
        enable_reasoning=False,
        prompt_cache_info=(False, None),
        streaming=True,
        tools=[rag_tool]
    )

# Wrap agent with StreamableAgent for queue-based streaming (Agent as a tool 사용할 경우, tool response 또한 스트리밍 하기 위해서)
agent = StreamableAgent(agent)

INFO:src.utils.strands_sdk_utils:RAG_AGENT - Prompt Cache Disabled


In [9]:
message = "만기 상환에 따른 수익률을 알려줄래?, 툴 결과를 받아서 정리하지말고 '완료' 라고만 말해줘"

# Stream with event queue pattern (like StreamableGraph.stream_async)
full_text = ""
async for event in agent.stream_async_with_queue(message):
    # Display events with colors
    strands_utils.process_event_for_display(event)
    
    # Collect text for response
    if event.get("event_type") == "text_chunk":
        full_text += event.get("data", "")

response = {"text": full_text}
print()  # New line after streaming


INFO [src.tools.rag_tool] ===== Executing RAG with Strands Agent =====

INFO [src.tools.rag_tool] ===== RAG - Query: 만기 상환에 따른 수익률 =====

INFO [src.tools.rag_tool] Bedrock Models Loaded - LLM: Claude-V4-5-Sonnet-CRI, Embedding: Cohere-Embed-V4-CRI

INFO [src.tools.rag_tool] OpenSearchHybridSearchRetriever initialized with advanced features


boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)



INFO [src.tools.rag_tool] Retrieved 5 documents, 2 tables, 1 images
INFO:src.utils.strands_sdk_utils:RAG_AGENT - Prompt Cache Disabled

INFO [src.tools.rag_tool] Adding 1 images to agent input

INFO [src.tools.rag_tool]   - Image 1: 74152 bytes

INFO [src.tools.rag_tool] Adding table 1 as image to agent input: 74152 bytes

INFO [src.tools.rag_tool] Adding table 2 as image to agent input: 62282 bytes

INFO [src.tools.rag_tool] Sending multimodal message with 4 content blocks


관련 내용을 찾았습니다.

**만기 상환에 따른 수익률:**

이 상품의 만기 상환 수익률은 두 가지 경우로 나뉩니다:

**① 만기평가가격이 최초기준가격의 100% 이상인 경우:**
- 만기수익률이 낮은 기초자산을 기준으로 계산
- 수익률 = (만기평가가격 - 최초기준가격) / 최초기준가격 × 70%
- 즉, 기초자산 상승률의 70%가 투자 수익률로 반영됩니다

**② 만기평가가격이 최초기준가격의 100% 미만인 경우:**
- 수익률 = 0%
- 원금 100% 보장으로 손실은 없지만 수익도 없습니다

**구체적인 사례:**
- **만기평가일 1**: KOSPI200 20% 상승, 금 10% 상승 → 낮은 수익률인 금(10%)을 기준으로 10% × 70% = **7.0% 수익률** (1억원 투자 시 700만원 수익)
- **만기평가일 2**: KOSPI200 20% 상승, 금 10% 하락 → 금이 하락했으므로 **0% 수익률** (원금만 보장)

핵심은 두 기초자산(KOSPI200, 금) 중 **수익률이 낮은 자산을 기준**으로 하며, 그 수익률의 **70%만 참여**한다는 점입니다.
[TOOL RESULT - rag_tool]
rag response:
만기 상환에 따른 수익률||관련 내용을 찾았습니다.

**만기 상환에 따른 수익률:**

이 상품의 만기 상환 수익률은 두 가지 경우로 나뉩니다:

**① 만기평가가격이 최초기준가격의 100% 이상인 경우:**
- 만기수익률이 낮은 기초자산을 기준으로 계산
- 수익률 = (만기평가가격 - 최초기준가격) / 최초기준가격 × 70%
- 즉, 기초자산 상승률의 70%가 투자 수익률로 반영됩니다

**② 만기평가가격이 최초기준가격의 100% 미만인 경우:**
- 수익률 = 0%
- 원금 100% 보장으로 손실은 없지만 수익도 없습니다

**구체적인 사례:**
- **만기평가일 1**: KOSPI200 20% 상승, 금 10% 상승 → 낮은 수익률인 금(10%)을 기준으로 10% × 70% = **7.0% 수익률** (